In [2]:
from sentence_transformers import CrossEncoder

reranker_model_name = 'jinaai/jina-reranker-v2-base-multilingual'
rerank_model = CrossEncoder(reranker_model_name, num_labels=1, max_length=1024, trust_remote_code=True)


/home/bsc/bsc830651/.conda/envs/factcheck/lib/python3.10/site-packages/flash_attn/ops/triton/layer_norm.py:959: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/home/bsc/bsc830651/.conda/envs/factcheck/lib/python3.10/site-packages/flash_attn/ops/triton/layer_norm.py:1018: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout, *args):


In [23]:
from torch.utils.data import DataLoader, Dataset
import torch
from sentence_transformers.readers import InputExample
from tqdm import tqdm

train_samples = [InputExample(texts=["Empty", "Full"], label=1)]
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=1)
train_dataloader.collate_fn = rerank_model.smart_batching_collate


for features, labels in tqdm(
            train_dataloader
        ):
    model_predictions = rerank_model.model(**features, return_dict=True)
    # logits = activation_fct(model_predictions.logits)
    # if self.config.num_labels == 1:
    # logits = logits.view(-1)
    # loss_value = loss_fct(logits, labels)
    # loss_value.backward()
    # torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_grad_norm)
    # optimizer.step()


100%|██████████| 1/1 [00:00<00:00, 149.06it/s]


In [25]:
model_predictions

SequenceClassifierOutput(loss=None, logits=tensor([[-0.7891]], device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [24]:
from torch import nn
sigmoid = nn.Sigmoid()
sigmoid(model_predictions.logits)

tensor([[0.3125]], device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<SigmoidBackward0>)